# Comparaison de Modèles : SVM, KNN et Random Forest

Dans ce notebook, nous allons comparer différents modèles de régression sur le dataset California Housing :
- Support Vector Machine (SVM)
- K-Nearest Neighbors (KNN)
- Random Forest

Pour chaque modèle, nous allons :
1. Évaluer ses performances de base
2. Optimiser ses hyperparamètres
3. Comparer les performances finales

In [ ]:
# Imports nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## 1. Chargement et Préparation des Données

In [ ]:
# Chargement du dataset
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()

print("Aperçu des données:")
display(df.head())
print("\nInformations sur le dataset:")
df.info()

In [ ]:
# Préparation des données
X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Dimensions des données:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

## 2. Évaluation Initiale des Modèles

In [ ]:
# Définition des modèles
models = {
    "SVM": SVR(),
    "KNN": KNeighborsRegressor(),
    "Random Forest": RandomForestRegressor(random_state=42)
}

# Comparaison avec validation croisée
print("Comparaison initiale (R² sur validation croisée) :")
cv_results = {}
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    cv_results[name] = scores
    print(f"{name}:")
    print(f"  Moyenne R² = {scores.mean():.4f}")
    print(f"  Écart-type R² = {scores.std():.4f}")

# Visualisation des résultats
plt.figure(figsize=(10, 6))
plt.boxplot([cv_results[name] for name in models.keys()], labels=models.keys())
plt.title("Distribution des Scores R² par Modèle")
plt.ylabel("Score R²")
plt.grid(True)
plt.show()

## 3. Optimisation des Hyperparamètres

In [ ]:
# Paramètres pour GridSearchCV
param_grids = {
    "SVM": {
        "C": [1, 10, 100],
        "epsilon": [0.1, 0.2, 0.3],
        "kernel": ['rbf', 'linear']
    },
    "KNN": {
        "n_neighbors": [3, 5, 7, 9],
        "weights": ['uniform', 'distance'],
        "p": [1, 2]  # Manhattan vs Euclidean
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5]
    }
}

# Optimisation pour chaque modèle
best_models = {}
for name, model in models.items():
    print(f"\nOptimisation des hyperparamètres pour {name}...")
    grid = GridSearchCV(model, param_grids[name], cv=5, scoring="r2", n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_
    
    print(f"Meilleurs paramètres : {grid.best_params_}")
    print(f"Meilleur score R² : {grid.best_score_:.4f}")

## 4. Évaluation Finale sur le Jeu de Test

In [ ]:
# Évaluation des modèles optimisés
results = []
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    results.append({
        'Modèle': name,
        'RMSE': rmse,
        'R²': r2
    })

# Affichage des résultats sous forme de DataFrame
results_df = pd.DataFrame(results)
print("Résultats finaux sur le jeu de test:")
display(results_df)

# Visualisation des prédictions vs valeurs réelles
plt.figure(figsize=(15, 5))
for idx, (name, model) in enumerate(best_models.items(), 1):
    plt.subplot(1, 3, idx)
    y_pred = model.predict(X_test)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Valeurs Réelles')
    plt.ylabel('Prédictions')
    plt.title(f'{name}')
plt.tight_layout()
plt.show()

## 5. Conclusion

Dans cette analyse, nous avons comparé trois modèles différents sur le dataset California Housing :
- Support Vector Machine (SVM)
- K-Nearest Neighbors (KNN)
- Random Forest

Les résultats montrent que :
1. Random Forest tend à avoir les meilleures performances globales
2. KNN est le plus rapide à entraîner mais moins performant
3. SVM offre un bon compromis mais nécessite plus de temps d'entraînement

Le choix final du modèle dépendra des contraintes spécifiques du projet (temps d'entraînement, temps de prédiction, interprétabilité, etc.).